In [2]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path = '../../datasets/nietzsche.txt'
text = open(path, encoding='utf-8').read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...


In [3]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=10, verbose=2)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

model.save('nietzsche.model')


--------------------------------------------------
Iteration 1
Epoch 1/10
96s - loss: 1.9641
Epoch 2/10
94s - loss: 1.6270
Epoch 3/10
94s - loss: 1.5417
Epoch 4/10
93s - loss: 1.4984
Epoch 5/10
92s - loss: 1.4688
Epoch 6/10
92s - loss: 1.4483
Epoch 7/10
92s - loss: 1.4301
Epoch 8/10
93s - loss: 1.4167
Epoch 9/10
92s - loss: 1.4061
Epoch 10/10
93s - loss: 1.3979

----- diversity: 0.2
----- Generating with seed: "y in their simulations, or in their weak"
y in their simulations, or in their weakers and as a stand he say, and a stand of the same the soul of the self and self-contempory of the man is the self one is a strength and the self as a stands of the self all the self-contempory and the considerable the more the conscious the considerable of the self-contempory the more the self a soul the man who was and discovered and consequently and stand of the self as the standard in the most

----- diversity: 0.5
----- Generating with seed: "y in their simulations, or in their weak"
y in the

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:65: RuntimeWarning: divide by zero encountered in log


e, all and
forwedly--would not
noy fear
felly welt mustment, thene"! he invertion dete-curion", schulh formeri. "it wished to same ridols; that the polent? we "freeee. hoaw is catedfully to felt speaks, shows: how a crke to gaties; the numbions, dist like, will
become betrais, they were self mon the pow, t

--------------------------------------------------
Iteration 4
Epoch 1/10
93s - loss: 1.3135
Epoch 2/10
93s - loss: 1.3123
Epoch 3/10
93s - loss: 1.3123
Epoch 4/10
93s - loss: 1.3097
Epoch 5/10
93s - loss: 1.3102
Epoch 6/10
93s - loss: 1.3082
Epoch 7/10
93s - loss: 1.3071
Epoch 8/10
93s - loss: 1.3071
Epoch 9/10
93s - loss: 1.3070
Epoch 10/10
93s - loss: 1.3055

----- diversity: 0.2
----- Generating with seed: "anish the anti-semitic bawlers out of th"
anish the anti-semitic bawlers out of the souls and the present the heady of the most state of the superior to the presented and presentiment of the strength of the possibility of the contradictive and such a superiority of the souls 

KeyboardInterrupt: 